In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
base_url = 'https://www.houzz.com/professionals/kitchen-and-bath-remodelers/miami-fl-us-probr0-bo~t_11825~r_4164138'
page_number = 0
lst = []

# Scrap to only first 3 pages in order to save time page count can be change to whatever any number of pages

while True and page_number <= 3:
    # Construct the URL for the current page
    if page_number == 0:
        url = base_url
    else:
        url = base_url + f'?fi={page_number*15}'
        
    # Send a GET request to the URL
    res = requests.get(url)
    
    # Break the loop if there are no more pages
    if 'No results found.' in res.text:
        break
    
    # Parse the HTML content of the response
    soup = BeautifulSoup(res.content, 'html.parser')
    
    # Find all the elements with class 'hz-pro-ctl' that are placed in a tag
    links = soup.find_all(lambda tag: tag.name == 'a' and 'hz-pro-ctl' in tag.get('class', []))
    
    # Extract the 'href' attribute of each link
    for link in links:
        href = link.get('href')
        lst.append(href)
        
    # Increment the page number for the next iteration
    page_number += 1

In [3]:
lsthead = lst[:4]
lsthead

['https://www.houzz.com/professionals/general-contractors/full-service-usa-pfvwus-pf~1830092161',
 'https://www.houzz.com/professionals/kitchen-and-bath-remodelers/orange-home-design-remodel-build-pfvwus-pf~188204162',
 'https://www.houzz.com/professionals/general-contractors/chapter-pfvwus-pf~666806768',
 'https://www.houzz.com/professionals/kitchen-and-bath-remodelers/construction-by-design-eg-pfvwus-pf~1136444938']

In [4]:
import re
names = []
phones = []
websites = []
adresses = []
for i in lsthead:
    res = requests.get(i)
    soup = BeautifulSoup(res.content, 'html.parser')    
    tag = soup.find_all("div", {"class": "sc-183mtny-0 sc-1uw6j8i-0 BusinessDetails__StyledCell-sc-1iscszt-0 dYJOPh ecpWHO gRCcss hui-cell"})
    for i, char in enumerate(tag):
        if i % 4 == 0:
            names.append(re.sub(r'^.*Business Name', '',char.text).strip())
        elif i % 4 == 1:
            phones.append(re.sub(r'^.*Phone Number', '', char.text).strip())
            
        elif i % 4 == 2:
            websites.append(re.sub(r'^.*Website', '', char.text).strip())
            
        else:
            adresses.append(re.sub(r'^.*Address', '', char.text).strip())

        if len(names) == len(phones) == len(websites) == len(adresses):
            break

In [5]:
print(adresses[:3])
print(phones[:3])
print(websites[:3])
print(names[:3])

['818 Northeast 8th StreetHallandale, Florida 33009United States', '1198 Venetian Way, 213Miami Beach, Florida 33139United States', '345 7th Ave 9th floorNew York, New York 10001United States']
['(305) 487-8485', '(305) 903-5158', '(212) 680-4628']
['www.fullserviceusa.com', 'orangehomedesign.com', 'hellochapter.com/']
['Full Service USA', 'Orange Home • Design | Remodel | Build', 'Chapter']


In [6]:
social_links = []
social_name = []

for i in lsthead:
    soc = []
    soclnks = []
    res = requests.get(i)
    soup = BeautifulSoup(res.text, "html.parser")
    a_tags = soup.find_all("a", {"class": "sc-62xgu6-0 jtYTMy sc-mwxddt-0 dOJEup hui-link"})
    soclnks.extend([a_tag.get("href") for a_tag in a_tags if a_tag.get("href")])
    soc.extend([a_tag.get("aria-label").split()[-1] for a_tag in a_tags if a_tag.get("href")])
    
    social_name.append(soc)
    social_links.append(soclnks)
    

In [7]:
for i in range(len(lsthead)):
    print(f"Houzz Url: {lsthead[0]} \n Name: {names[i]} \n Phone: {phones[i]} \n website url: {websites[i]} \n Adress:{adresses[i]}")
    for j in range (len(social_links[i])):
        print(f"{social_name[i][j]}: {social_links[i][j]}")
    print("---------------------------------------------------")

Houzz Url: https://www.houzz.com/professionals/general-contractors/full-service-usa-pfvwus-pf~1830092161 
 Name: Full Service USA 
 Phone: (305) 487-8485 
 website url: www.fullserviceusa.com 
 Adress:818 Northeast 8th StreetHallandale, Florida 33009United States
site: https://www.houzz.com/trk/d3d3LmZ1bGxzZXJ2aWNldXNhLmNvbQ/ab5d3a8e97b7ce05169f22f794bef98c/ue/MjI1OTQ1Mzc/30f4d162afc94c9405b53cba2d53f2c3
---------------------------------------------------
Houzz Url: https://www.houzz.com/professionals/general-contractors/full-service-usa-pfvwus-pf~1830092161 
 Name: Orange Home • Design | Remodel | Build 
 Phone: (305) 903-5158 
 website url: orangehomedesign.com 
 Adress:1198 Venetian Way, 213Miami Beach, Florida 33139United States
Facebook: https://www.houzz.com/trk/aHR0cHM6Ly93d3cuZmFjZWJvb2suY29tL29yYW5nZWhvbWVkZXNpZ24v/5d1d65e98ecb253497482bca949b89a5/ue/NTczNjU0NjY/a6e5762785772e368f4bf16ed6319722
Twitter: https://www.houzz.com/trk/aHR0cHM6Ly93d3cudHdpdHRlci5jb20vb3JhbmdlaG9tZWRz

In [8]:
import csv

soc_col = []

with open('data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Houzz Url','Name','Phone','Website','Adress'])
    for row in zip(lsthead, names, phones,websites,adresses):
        writer.writerow(row)         